In [1]:
mode = "png"

import matplotlib

font = {'family' : 'Dejavu Sans',
        'weight' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)

import matplotlib
from matplotlib import pyplot as plt

In [2]:


fig.tight_layout()
fig.savefig("Figures/.{}".format(mode))

NameError: name 'fig' is not defined

In [ ]:
fig.tight_layout()
fig.savefig("Figures/.{}".format(mode))

In [ ]:
fig.tight_layout()
fig.savefig("Figures/.{}".format(mode))

In [ ]:
fig.tight_layout()
fig.savefig("Figures/.{}".format(mode))